In [15]:
import os
import time
import random
from PIL import Image
from skimage import io
from skimage.transform import resize
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from pathlib import Path
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt

MAE = tf.keras.losses.MeanAbsoluteError()
MSE = tf.keras.losses.MeanSquaredError()

In [2]:
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
model = hub.load(SAVED_MODEL_PATH)

In [3]:
def png(image, in_path, out_path):
    band = np.load(in_path + image)
    GSband = np.zeros((band.shape[0], band.shape[1], 3))
    GSband[:,:,0]= band
    GSband[:,:,1]= band
    GSband[:,:,2]= band
    GSband = GSband.astype('uint8')
    png = Image.fromarray(GSband)
    image = image[:-4]
    png.save(out_path + image + ".png")

def random_pngs():
path = '../../GOES_Files/diff_files/'
for x in range(20):
    file = random.choice(os.listdir(path))
    png(file, path, '../../GOES_Files/png_files/')

In [14]:
# random_pngs()

In [42]:
def preprocess_image(image_path):
    #need to divide values
    hr_image = tf.image.decode_image(tf.io.read_file(image_path))
    if hr_image.shape[-1] == 4:
        hr_image = hr_image[...,:-1]
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 2) * 2
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)

def downscale_image(image):
    image_size = []
    if len(image.shape) == 3:
        image_size = [image.shape[1], image.shape[0]]
    else:
        raise ValueError("Dimension mismatch. Can work only on single image.")

    image = tf.squeeze(tf.cast(tf.clip_by_value(image, 0, 255), tf.uint8))

    lr_image = np.asarray(
#     Image.fromarray(image.numpy()).resize([image_size[0] // 4, image_size[1] // 4], Image.BICUBIC))
    Image.fromarray(image.numpy()).resize([image_size[0] // 2, image_size[1] // 2], Image.NEAREST))

    lr_image = tf.expand_dims(lr_image, 0)
    lr_image = tf.cast(lr_image, tf.float32)
    return lr_image


def m_learning(fileID):
    band = fileID[-58:-56]
    hr_image = preprocess_image(fileID)
    hr_image = tf.squeeze(hr_image)
    lr_image = downscale_image(hr_image)
    fake_image = model(lr_image)
    fake_image = tf.squeeze(fake_image)
    fake_image = resize(fake_image, (500, 500, 3))
    lr_image = resize(tf.squeeze(lr_image), (500, 500, 3))

#     controlmae = np.sum(resize(tf.squeeze(lr_image), (500, 500, 3)) - hr_image)/(500*500)
#     controlrmse = np.sqrt(np.sum((resize(tf.squeeze(lr_image), (500, 500, 3)) - hr_image)**2)/(500*500))
#     mae = np.sum(fake_image - hr_image)/(500*500)
#     rmse = np.sqrt(np.sum(fake_image - hr_image)**2)/(500*500)
    
    controlmae = MAE(hr_image, lr_image).numpy()
    controlrmse = np.sqrt(MSE(hr_image, lr_image).numpy())
    mae = MAE(hr_image, fake_image).numpy()
    rmse = np.sqrt(MSE(hr_image, fake_image).numpy())
    
    table.append({'File': fileID, 'Control MAE': controlmae, 'Control RMSE': controlrmse, 'MAE': mae, 'RMSE': rmse})

In [43]:
table = []
path = '../../GOES_Files/png_files/'
for file in os.listdir(path):
    try:
        m_learning(path + file)
    except:
        print(file)

.ipynb_checkpoints


In [44]:
x = pd.DataFrame(table)
agg_function = {'Control MAE': 'mean', 'Control RMSE': 'mean', 'MAE': 'mean', 'RMSE': 'mean'}
y = x.agg(agg_function)
y

Control MAE      5.551455
Control RMSE    20.539092
MAE              9.448898
RMSE            30.639948
dtype: float64